In [1]:
# !pip install notebook --upgrade

In [2]:
# !jupyter colab --NotebookApp.iopub_data_rate_limit=10000000000

In [3]:
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 14.0 MB/s eta 0:00:00
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 21 not upgraded.
Need to get 36.6 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 124016 files and directories currently installed.)
Pre

In [4]:
# from tqdm import tqdm
# import numpy as np
# from collections import Counter
# from threading import Thread

# import inverted_index_gcp
import inverted_index_gcp
import math
import pickle
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *
import time
from inverted_index_gcp import MultiFileReader, InvertedIndex
import struct
import time
import os
from google.cloud import storage
from contextlib import closing
import pickle as pkl

import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
# from time import time
# from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 KB 3.5 MB/s eta 0:00:00


In [6]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()


In [7]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth
## RENAME the project_id to yours project id from the project you created in GCP 
project_id = 'myfirstgcp-370210'
!gcloud config set project {project_id}

data_bucket_name = 'elad_318640828_titles_bucket'
try:
    if os.environ["elad_318640828_titles_bucket"] is not None:
        pass  
except:
      !mkdir wikidumps
      # !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 

Updated property [core/project].


In [8]:
# !gsutil cp -R gs://elad_318640828_titles_bucket/small_corpus .
!gsutil -m cp -R gs://elad_318640828_titles_bucket/small_postings_gcp .

Copying gs://elad_318640828_titles_bucket/small_postings_gcp/104_posting_locs.pickle...
Copying gs://elad_318640828_titles_bucket/small_postings_gcp/104_000.bin...
Copying gs://elad_318640828_titles_bucket/small_postings_gcp/28_000.bin...
Copying gs://elad_318640828_titles_bucket/small_postings_gcp/56_000.bin...
Copying gs://elad_318640828_titles_bucket/small_postings_gcp/28_posting_locs.pickle...
Copying gs://elad_318640828_titles_bucket/small_postings_gcp/index_title.pkl...
Copying gs://elad_318640828_titles_bucket/small_postings_gcp/56_posting_locs.pickle...


In [9]:
import pickle

# Replace <path/to/dictionary.pkl> with the path to your dictionary file
with open('./small_postings_gcp/index_title.pkl', 'rb') as f:
    index_anchor = pickle.load(f)



In [10]:
print(index_anchor.df["world"])

60


In [11]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124


def token2bucket_id(token):
    return int(_hash(token), 16) % NUM_BUCKETS

In [12]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    # print(inverted.posting_locs[w])
    locs = inverted.posting_locs[w]
    # print(locs)
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    # print(b)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [13]:
query = "world cUp"
def search_title(query):
    ''' Returns up to a 100 search results for the query using TFIDF AND COSINE
        SIMILARITY OF THE BODY OF ARTICLES ONLY. DO NOT use stemming. DO USE the 
        staff-provided tokenizer from Assignment 3 (GCP part) to do the 
        tokenization and remove stopwords. 

        To issue a query navigate to a URL like:
         http://YOUR_SERVER_DOMAIN/search_body?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of up to 100 search results, ordered from best to worst where each 
        element is a tuple (wiki_id, title).
    '''
    res = []
    # query = request.args.get('query', '')
    # if len(query) == 0:
    #   return jsonify(res)
  # BEGIN SOLUTION
    dic = {}
    tokens = [token.group() for token in RE_WORD.finditer(query.lower())]
    for term in tokens:
        if index_anchor.df.get(term):
            ls_doc_freq = read_posting_list(index_anchor,term)
            # print(ls_doc_freq)
            for doc, freq in ls_doc_freq:
              # print(doc)
              dic[doc] = dic.get(doc, 0) + freq
    lst_doc = Counter(dic).most_common()
    for item in lst_doc:
      print(item)
      res.append((item[0], index_anchor.id2titles[item[0]]))
  # END SOLUTION
    return res

In [14]:
search_title(query)

(11370, 2)
(16383, 2)
(33727, 2)
(59707, 2)
(168079, 2)
(183628, 2)
(244862, 2)
(656933, 2)
(1166428, 2)
(1248592, 2)
(1853149, 2)
(2150801, 2)
(2996777, 2)
(3482503, 2)
(3556431, 2)
(4723188, 2)
(4743361, 2)
(8258172, 2)
(8734046, 2)
(8821389, 2)
(10822574, 2)
(13327177, 2)
(16842834, 2)
(16966712, 2)
(17742072, 2)
(19537336, 2)
(22230053, 2)
(26814387, 2)
(27007503, 2)
(27226732, 2)
(29868391, 2)
(32352129, 2)
(32516422, 2)
(36581929, 2)
(39302261, 2)
(39812824, 2)
(41648358, 2)
(42931572, 2)
(45271353, 2)
(51765484, 2)
(55490096, 2)
(57240806, 2)
(57918704, 2)
(57918711, 2)
(60410401, 2)
(60637832, 2)
(61269058, 2)
(61715824, 2)
(62528055, 2)
(64467696, 2)
(64999764, 2)
(64999924, 2)
(66040084, 2)
(66040086, 2)
(67608822, 2)
(22888933, 1)
(29384326, 1)
(36827305, 1)
(42072639, 1)
(63946361, 1)


[(11370, 'FIFA World Cup'),
 (16383, 'FIFA World Cup Trophy'),
 (33727, 'World cup'),
 (59707, '1930 FIFA World Cup'),
 (168079, '2010 FIFA World Cup'),
 (183628, 'FIS Alpine Ski World Cup'),
 (244862, "FIFA Women's World Cup"),
 (656933, '2014 FIFA World Cup'),
 (1166428, 'FIFA World Cup official mascots'),
 (1248592, 'FIFA Club World Cup'),
 (1853149, 'FIFA World Cup video games'),
 (2150801, '2018 and 2022 FIFA World Cup bids'),
 (2996777, 'FIFA Futsal World Cup'),
 (3482503, 'FIFA World Cup awards'),
 (3556431, 'FIFA World Cup qualification'),
 (4723188, 'History of the FIFA World Cup'),
 (4743361, 'FIFA World Cup hosts'),
 (8258172, "ICC Men's T20 World Cup"),
 (8734046, 'Brazil at the FIFA World Cup'),
 (8821389, 'List of FIFA World Cup finals'),
 (10822574, 'List of FIFA World Cup broadcasters'),
 (13327177, '2018 FIFA World Cup'),
 (16842834, 'World Cup (disambiguation)'),
 (16966712, 'Argentina at the FIFA World Cup'),
 (17742072, '2022 FIFA World Cup'),
 (19537336, '1930 FIFA

In [15]:
index_anchor.id2titles[38310]

'Cannabis'

In [ ]:
query = "world cUp"
def search_body(query):
    ''' Returns up to a 100 search results for the query using TFIDF AND COSINE
        SIMILARITY OF THE BODY OF ARTICLES ONLY. DO NOT use stemming. DO USE the 
        staff-provided tokenizer from Assignment 3 (GCP part) to do the 
        tokenization and remove stopwords. 

        To issue a query navigate to a URL like:
         http://YOUR_SERVER_DOMAIN/search_body?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of up to 100 search results, ordered from best to worst where each 
        element is a tuple (wiki_id, title).
    '''
    res = []

    with_stop_words_query_tokens = [token.group() for token in RE_WORD.finditer(query.lower())]

    query_tokens = []
    for tok in with_stop_words_query_tokens:
        if tok not in all_stopwords:
            query_tokens.append(tok)
    query_tokens = Counter(query_tokens)

    # this is normaliztion of the query for the cosine similarity
    norm_factor = 0
    for term, freq in query_tokens.items():
      norm_factor += freq**2
    norm_factor = math.sqrt(norm_factor)
    # print(norm_factor)
    if norm_factor == 0:
      raise ValueError("you insert an empty query")

    sim_dict = {}
    for term, freq in query_tokens.items():
      try:
        posting_list_per_term = read_posting_list(index_body, term)
        print(posting_list_per_term)
      except:
        continue
      norm_factor_doc = 0
      for doc_id, score in posting_list_per_term:
        if doc_id in sim_dict:
          sim_dict[doc_id] += score
        else:
          sim_dict[doc_id] = score
      for doc_id in sim_dict.keys():
        norm_factor_doc += sim_dict[doc_id]**2
      norm_factor_docs_final = math.sqrt(norm_factor_doc)
      if norm_factor_docs_final == 0:
        raise ValueError("You need to check your '.bin' files maybe they are not include in the folder")
    similarity_final_dict = dict(sorted(sim_dict.items(), key= lambda x: (x[1]/(norm_factor * norm_factor_docs_final)), reverse= True))

    iteration = 0
    for doc_id in similarity_final_dict.keys():
      if iteration < 100:
        res.append((doc_id, index_body.id2titles[doc_id]))
        iteration += 1
      break
    return res



print(search_body(query))

[]
